In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## _Sorting out cells that made it to landfall (used the globe.is_land() library)_

In [2]:
track = pd.read_csv('../saved-files/threshold-10/final-tracks-threshold-10.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,4,112.472423,381.827530,14,10,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,4,0 days 00:00:00,133.652175,13.245867,34.567482,22.880043
1,1,4,113.727184,383.443874,16,10,105,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,4,0 days 00:05:00,133.177717,12.974534,35.577388,22.881160
2,2,4,114.912344,385.528075,9,10,211,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,4,0 days 00:10:00,133.000505,12.780750,34.063904,15.916738
3,3,4,116.591310,387.338867,9,10,315,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,4,0 days 00:15:00,132.299723,12.708913,36.584485,14.922242
4,4,4,118.277261,388.995062,5,10,415,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,4,0 days 00:20:00,131.563070,12.845599,38.092506,9.948374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17189,299,37,344.225457,230.927378,85,10,24599,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.404225e+06,147927.378233,68.294564,13.520861,8480,0 days 00:10:00,145.295316,15.240493,41.116443,138.320857
17190,300,40,345.289261,230.420788,113,10,24640,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.405289e+06,147420.787679,68.285155,13.507285,8480,0 days 00:15:00,146.446244,14.702386,40.106802,146.281856
17191,298,4,144.157813,260.357823,215,10,24525,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.204158e+06,177357.823188,70.094551,14.600400,8481,0 days 00:00:00,109.386079,15.301029,39.603226,383.006996
17192,299,7,145.597643,261.961765,216,10,24569,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.205598e+06,178961.765093,70.080315,14.638806,8481,0 days 00:05:00,107.283607,15.240493,41.116443,396.941301


In [3]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,4,112.472423,381.827530,14,10,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,4,0 days 00:00:00,133.652175,13.245867,34.567482,22.880043
1,1,4,113.727184,383.443874,16,10,105,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,4,0 days 00:05:00,133.177717,12.974534,35.577388,22.881160
2,2,4,114.912344,385.528075,9,10,211,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.174912e+06,302528.074991,70.234954,17.918971,4,0 days 00:10:00,133.000505,12.780750,34.063904,15.916738
3,3,4,116.591310,387.338867,9,10,315,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.176591e+06,304338.867039,70.217548,17.959698,4,0 days 00:15:00,132.299723,12.708913,36.584485,14.922242
4,4,4,118.277261,388.995062,5,10,415,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.178277e+06,305995.062270,70.200267,17.996341,4,0 days 00:20:00,131.563070,12.845599,38.092506,9.948374


In [4]:
%%time
cell_landfall = []
for i, j in enumerate(track['cell']):
#     # # Check if a point is on land:
    lat = track['latitude'][i]
    lon = track['longitude'][i]
    is_on_land = globe.is_land(lat, lon)
    if is_on_land:
        # if ((lat >= 69.0) & (lon >= 12.0)):
            # print(f"lat={lat}, lon={lon} is on land: {is_on_land} -- cell_id={j}")
        cell_landfall.append(j)

CPU times: user 1.7 s, sys: 3.84 ms, total: 1.71 s
Wall time: 1.81 s


In [5]:
%%time
cells_landfall=np.unique(cell_landfall)
cells_landfall

CPU times: user 224 µs, sys: 24 µs, total: 248 µs
Wall time: 252 µs


array([  67,   68,   78,  110,  144,  176,  216,  253,  255,  263,  354,
        362,  368,  393,  401,  440,  442,  473,  477,  524,  526,  594,
        608,  611,  619,  629,  651,  665,  695,  700,  723,  725,  759,
        763,  766,  769,  773,  816,  848,  853,  893,  911,  914,  955,
        956,  959,  964,  969, 1000, 1006, 1011, 1016, 1045, 1109, 1111,
       1126, 1159, 1162, 1177, 1256, 1264, 1266, 1279, 1303, 1309, 1334,
       1337, 1340, 1358, 1360, 1370, 1410, 1418, 1458, 1460, 1477, 1484,
       1508, 1516, 1524, 1553, 1565, 1576, 1603, 1623, 1651, 1652, 1663,
       1664, 1668, 1704, 1709, 1735, 1739, 1740, 1747, 1750, 1771, 1785,
       1790, 1792, 1793, 1802, 1809, 1826, 1853, 1885, 1974, 1992, 2025,
       2029, 2042, 2050, 2052, 2062, 2081, 2088, 2091, 2129, 2133, 2142,
       2151, 2162, 2178, 2186, 2246, 2278, 2307, 2314, 2327, 2349, 2362,
       2390, 2401, 2485, 2509, 2519, 2569, 2607, 2675, 2682, 2690, 2795,
       2799, 2808, 2834, 2846, 2878, 2879, 2880, 28

## _Algorithm to track cells back in time that made it to landfall_

In [6]:
%%time
combined_cells_landfalls = []
for i, cell_ids in enumerate(np.unique(cells_landfall)): # looped through tracks found with AMF1 site
    if cell_ids in track['cell'].values: # checking if those cells(tracks) are within the original tracked dataframe
        # print(f'Cell {cell_ids} present: made it to landfall 😀')
        track_c = track[track['cell'] == cell_ids] # display it if it is within the tracked dataframe
        combined_cells_landfalls.append(track_c) # append the results you get a new dataFrame calld combined_tracks_AMF1
    else:
        print('Cells not present 🥲')
        
# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
combined_cells_landfall_present = pd.concat(combined_cells_landfalls)
combined_cells_landfall_present = combined_cells_landfall_present.reset_index(drop=True)
combined_cells_landfall_present['landfall'] = globe.is_land(combined_cells_landfall_present['latitude'], combined_cells_landfall_present['longitude'])

CPU times: user 210 ms, sys: 9.78 ms, total: 219 ms
Wall time: 242 ms


In [7]:
combined_cells_landfall_present.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall
0,0,67,326.578302,280.675950,5,10,67,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.386578e+06,197675.949726,68.422543,14.734897,67,0 days 00:00:00,104.413246,13.245867,34.567482,12.937082,True
1,1,68,325.560542,282.342019,49,10,169,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.385561e+06,199342.018987,68.430539,14.776648,67,0 days 00:05:00,102.818375,12.974534,35.577388,58.710257,True
2,2,71,326.725123,282.815677,49,10,278,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.386725e+06,199815.677294,68.419587,14.785620,67,0 days 00:10:00,103.725137,12.780750,34.063904,58.711762,True
3,0,68,336.121109,286.415064,196,10,68,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.396121e+06,203415.064277,68.331369,14.852404,68,0 days 00:00:00,111.375543,13.245867,34.567482,187.081946,True
4,1,76,335.309757,288.487718,148,10,177,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.395310e+06,205487.718307,68.337139,14.903259,68,0 days 00:05:00,109.948778,12.974534,35.577388,223.899986,True


In [8]:
len(combined_cells_landfall_present)

3334

## _track up to the time the cell first hit landfall (after it hits the first landfall, remove all the subsequent cells hitting landfall, we are not interested in it)_

In [9]:
%%time
cells_to_first_landfall = []
for i, cell_idx in enumerate(np.unique(cells_landfall)):
    df = combined_cells_landfall_present[combined_cells_landfall_present['cell'] == cell_idx]
    # # find the index of the first occurrence of True
    idx = df.loc[df['landfall']].index[0]
    # slice the DataFrame to keep only the rows up to and including that index
    df = df.loc[:idx]
    # set all subsequent occurrences of True to False
    df.loc[df['landfall'].shift(-1) == True, 'landfall'] = False
    # print(df)
    cells_to_first_landfall.append(df)
first_landfall = pd.concat(cells_to_first_landfall)

CPU times: user 620 ms, sys: 9.92 ms, total: 630 ms
Wall time: 680 ms


## _calculate relative time --- relative time = time - time_landfall_

In [10]:
%%time
cells_relative_time = []
for index, cell_id in enumerate(np.unique(first_landfall['cell'])):
    if cell_id:
        # print(cell_id)
        cells = first_landfall[first_landfall['cell'] == cell_id] # get all the cell_ids
    # calculate for the relative time ----- relative time = time - time_landfall
        all_cells_times = cells['timestr']; all_cells_times = pd.to_datetime(all_cells_times.values) # convert timestr to datetime
        landfall_time_each_cell = all_cells_times[::-1][0]
        # relative_time = np.absolute(all_cells_times - landfall_time_each_cell) # relative time
        relative_time = np.subtract(landfall_time_each_cell, all_cells_times)
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time.append(relative_time)

CPU times: user 450 ms, sys: 3.96 ms, total: 454 ms
Wall time: 487 ms


In [11]:
cells_relative_time[:20]

[Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([5, 0], dtype='int64'),
 Int64Index([30, 25, 20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([10, 5, 0], dtype='int64')]

In [12]:
# cells_relative_time
cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
# first_landfall['relative_time'] = cells_relative_times
# cells_relative_times.values
first_landfall['relative_time'] = cells_relative_times.values

In [13]:
len(first_landfall)

1381

In [14]:
first_landfall.to_csv('../saved-files/threshold-10/all-cells-first_landfall-threshold-10.csv', index=False)

In [15]:
# final = first_landfall[['cell', 'latitude', 'longitude', 'distance', 'max dBZ','mean dBZ', 'area' ,'timestr', 'landfall']]
# final

# END 